In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchRRelu1D,GRU_NCL
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss,RReLU
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalize(train[:,0])
# without tanh is worse-off

In [3]:
## change ##
batch_size = 16
clip = .25

Prepare data

In [4]:
train_gen = SequenceGen(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGen(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [5]:
class DenseBlock2(nn.Module):
    # N,D,L to N,2*D,L
    def __init__(self, in_channel):
        super().__init__()
        self.in_channel = in_channel
        self.conv1 = ConvBatchRRelu1D(in_channel,in_channel,3,padding=1)
        self.conv2 = ConvBatchRRelu1D(in_channel*2,in_channel*2,3,stride=2)
        
    def forward(self, x):
        return self.conv2(torch.cat([x,self.conv1(x)],1))
    
convNets = Sequential(ConvBatchRRelu1D(1,2,5,stride=2),
                      ConvBatchRRelu1D(2,4,5,stride=2),
                      ConvBatchRRelu1D(4,8,5,stride=2),
                      ConvBatchRRelu1D(8,16,5,stride=2),
                      ConvBatchRRelu1D(16,32,5,stride=2),
                      DenseBlock(32),
                      DenseBlock(64),
                      DenseBlock(128),
                      DenseBlock(256),
                      GRU_NCL(512,512,batch_first=True,bidirectional=True,returnH=True))

In [6]:
linear = Sequential(Linear(512*2,128),RReLU(True),Linear(128,1))

In [7]:
model = CNN_RNN2seq(convNets,linear).to('cuda:0')

In [8]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [9]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [10]:
model = fit(8, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.555333460179659, val_loss:2.4315590858459473
epoch:1, train_loss:2.308449317629521, val_loss:2.660294771194458
epoch:2, train_loss:2.3019992812321735, val_loss:2.454993724822998
epoch:3, train_loss:2.2914601805118413, val_loss:2.4253551959991455
epoch:4, train_loss:2.230441803542467, val_loss:2.429539918899536


Traceback (most recent call last):
  File "/home/will/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/will/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/will/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/will/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/will/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/will/anaconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/will/anaconda3/envs/pytorch/lib/

KeyboardInterrupt: 